In [1]:
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse

In [2]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [01:47, 1867.79it/s]


#  Домашнее задание по NLP # 1 [100 баллов]
## Классификация по тональности 

В этом домашнем задании вам предстоит классифицировать по тональности отзывы на банки с сайта banki.ru.

Данные содержат непосредственно тексты отзывов, некоторую дополнительную информацию, а также оценку по шкале от 1 до 5. 

Тексты хранятся в json-ах в массиве responses.

Посмотрим на пример отзыва:

## Часть 1. Анализ текстов [40/100]

### 1. Посчитайте количество отзывов в разных городах и на разные банки

In [ ]:
banks={}
cities={}
for i in range(0,len(responses)):
    if responses[i].get('bank_name') in banks.keys():
        banks[responses[i].get('bank_name')]+=1
    else:
        banks[responses[i].get('bank_name')]=1
    if responses[i].get('city') in cities.keys():
        cities[responses[i].get('city')]+=1
    else:
        cities[responses[i].get('city')]=1
banks = sorted(banks.items(), key=lambda kv: kv[1],reverse=True)
cities= sorted(cities.items(), key=lambda kv: kv[1],reverse=True)

In [ ]:
banks[:5]

In [ ]:
cities[:5]

### 2. Постройте гистограмы длин слов в символах и в словах

In [4]:
import re
regex = re.compile("[А-Яа-я]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""

In [ ]:
reviews_symb = []
reviews_word = []
for i in tqdm(range(0,len(responses))):
    reviews_symb.append(len(responses[i].get('text')))
    reviews_word.append(len(words_only(responses[i].get('text')).split(' ')))

In [ ]:
plt.hist(reviews_symb);
plt.title("Гистограммма по кол-ву символов в отзыве");
plt.xlabel("Кол-во символов в отзыве");

In [ ]:
plt.hist(reviews_word);
plt.title("Гистограммма по кол-ву слов в отзыве");
plt.xlabel("Кол-во слов в отзыве");

### 3. Найдите 10 самых частых:
  * слов
  * слов без стоп-слов
  * лемм 
  * существительных


In [ ]:
from collections import Counter


In [5]:
reviews_df = pd.DataFrame(responses)

In [6]:
reviews_df['text']=reviews_df.text.str.lower().apply(words_only)

In [7]:
def the_most_common_words(df):
    cnt = Counter()
    tokens = []
    tokens = df.text.apply(lambda row: cnt.update(row.split(' ')))
    return cnt     

In [ ]:
for i in the_most_common_words(reviews_df).most_common(10):
    print(i)

In [8]:
from nltk.corpus import stopwords
mystopwords = stopwords.words('russian')

In [9]:
def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return "" 

In [10]:
reviews_df['text']=reviews_df.text.apply(remove_stopwords)

In [ ]:
for i in the_most_common_words(reviews_df).most_common(10):
    print(i)

In [13]:
from pymystem3 import Mystem
mystem = Mystem()

In [14]:
from multiprocessing import Pool

In [23]:
def def_lemm(text):
    try:
        return "".join(mystem.lemmatize(text)).strip() 
    except:
        return " "

def lemmatize(df, mystem=mystem):
    df['text']=df.text.apply(def_lemm)
    return df
    

In [26]:
%%timeit
lemmatize(reviews_df[:8])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


7.59 s ± 40.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
from multiprocessing import Pool
num_partitions = 10 #number of partitions to split dataframe
num_cores = 8 #number of cores on your machine

In [17]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
%%timeit
reviews_df1= parallelize_dataframe(reviews_df[:8], lemmatize)

In [ ]:
data = parallelize(reviews_df.text, lemmatize)

In [ ]:
reviews_df['text'] = reviews_df.text.progress_apply(lemmatize)

In [ ]:
lemmatize(df.text)

In [ ]:
lemmas2 = mystem.lemmatize(all_reviews[0])
print(''.join(lemmas2))

### 4. Постройте кривые Ципфа и Хипса

In [ ]:
all_reviews[1]

### 5. Ответьте на следующие вопросы:
   * какое слово встречается чаще, "сотрудник" или "клиент"?
   * сколько раз встречается слова "мошенничество" и "доверие"?


### 6. В поле "rating_grade" записана оценка отзыва по шкале от 1 до 5. Используйте меру $tf-idf$, для того, чтобы найти ключевые слова и биграмы для положительных отзывов (с оценкой 5) и отрицательных отзывов (с оценкой 1)

## Часть 2. Тематическое моделирование [20/100]

1. Постройте несколько тематических моделей коллекции документов с разным числом тем. Приведите примеры понятных (интерпретируемых) тем.
2. Найдите темы, в которых упомянуты конкретные банки (Сбербанк, ВТБ, другой банк). Можете ли вы их прокомментировать / объяснить?

Эта часть задания может быть сделана с использованием gensim. 

## Часть 3. Классификация текстов [40/100]

Сформулируем для простоты задачу бинарной классификации: будем классифицировать на два класса, то есть, различать резко отрицательные отзывы (с оценкой 1) и положительные отзывы (с оценкой 5). 

1.  Составьте обучающее и тестовое множество: выберите из всего набора данных N1 отзывов с оценкой 1 и N2 отзывов с оценкой 5 (значение N1 и N2 – на ваше усмотрение). Используйте ```sklearn.model_selection.train_test_split``` для разделения множества отобранных документов на обучающее и тестовое. 
2. Используйте любой известный вам алгоритм классификации текстов для решения задачи и получите baseline. Сравните разные варианты векторизации текста: использование только униграм, пар или троек слов или с использованием символьных $n$-грам. 
3. Сравните, как изменяется качество решения задачи при использовании скрытых тем в качестве признаков:
* 1-ый вариант: $tf-idf$ преобразование (```sklearn.feature_extraction.text.TfidfTransformer```) и сингулярное разложение (оно же – латентый семантический анализ) (```sklearn.decomposition.TruncatedSVD```), 
* 2-ой вариант: тематические модели LDA (```sklearn.decomposition.LatentDirichletAllocation```). 

Используйте accuracy и F-measure для оценки качества классификации. 

Ниже написан примерный Pipeline для классификации текстов. 

Эта часть задания может быть сделана с использованием sklearn. 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# !!! На каждом этапе Pipeline нужно указать свои параметры
# 1-ый вариант: tf-idf + LSI
# 2-ой вариант: LDA

# clf = Pipeline([
#     ('vect', CountVectorizer(analyzer = 'char', ngram_range={4,6})),
#     ('clf', RandomForestClassifier()),
# ])



clf = Pipeline([ 
    ('vect', CountVectorizer()), 
    ('tfidf', TfidfTransformer()), 
    ('tm', TruncatedSVD()), 
    ('clf', RandomForestClassifier())
])
